In [1]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
# Binary to Integer Generator
def getInt(b_list):
    binary = ""
    for i in b_list:
        binary += str(int(i))
    return int(binary, 2)

# Test case
binary_number = [1, 0, 1]
print getInt(binary_number)

5


In [3]:
# Dataset Creation
def create_data(num_samples, len_binary):
    x = np.zeros((num_samples, len_binary))
    y = np.zeros(num_samples)
    for i in range(num_samples):
        x[i] = np.round(np.random.rand(len_binary)).astype(int)
        y[i] = getInt(x[i])
    return x, y

# Test Case
X, y = create_data(5, 4)
for i in range(X.shape[0]):
    print X[i], '\t ', y[i]

[ 1.  1.  0.  0.] 	  12.0
[ 1.  0.  0.  0.] 	  8.0
[ 1.  0.  0.  1.] 	  9.0
[ 0.  0.  1.  1.] 	  3.0
[ 0.  1.  1.  0.] 	  6.0


In [4]:
# TF Model Parameters
binary_length = 8
training_samples = 1000
testing_samples = 20
lr = 0.01
training_steps = 50000
display_steps = 5000
n_input = binary_length
n_hidden_units = 16
n_output = 1
timestep = 1

In [5]:
# Generate Training and Testing Data
X_train, y_train = create_data(training_samples, n_input)
X_test, y_test = create_data(testing_samples, n_input)

# Print data
display = 5
for i in range(display):
    print X_train[i], '\t', y_train[i], "\n"

[ 1.  0.  0.  1.  1.  0.  1.  0.] 	154.0 

[ 0.  1.  0.  1.  1.  1.  0.  0.] 	92.0 

[ 1.  1.  0.  0.  0.  0.  1.  1.] 	195.0 

[ 1.  1.  1.  0.  1.  1.  0.  0.] 	236.0 

[ 1.  1.  0.  1.  0.  0.  1.  0.] 	210.0 



In [6]:
# TF Model and intializations
X = tf.placeholder(tf.float32, [None, timestep, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
W = tf.Variable(tf.random_normal([n_hidden_units, n_output]))
b = tf.Variable(tf.random_normal([n_output]))

def model(X, W, b, timestep, n_hidden_units):
    X = tf.unstack(X, timestep, 1)
    lstm_cell = rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, X, dtype=tf.float32)
    logits = tf.matmul(outputs[-1], W) + b
    return logits

logits = model(X, W, b, timestep, n_hidden_units)
loss = tf.reduce_mean(tf.losses.mean_squared_error(logits, y))
optimizer = tf.train.RMSPropOptimizer(lr)
training = optimizer.minimize(loss)

In [7]:
# Reshape data
X_train = np.reshape(X_train, [-1, timestep, n_input])
y_train = np.reshape(y_train, [-1, n_output])

X_test = np.reshape(X_test, [-1, timestep, n_input])
y_test = np.reshape(y_test, [-1, n_output])

# Print data
display = 5
for i in range(display):
    print X_train[i], '\t', y_train[i]

[[ 1.  0.  0.  1.  1.  0.  1.  0.]] 	[ 154.]
[[ 0.  1.  0.  1.  1.  1.  0.  0.]] 	[ 92.]
[[ 1.  1.  0.  0.  0.  0.  1.  1.]] 	[ 195.]
[[ 1.  1.  1.  0.  1.  1.  0.  0.]] 	[ 236.]
[[ 1.  1.  0.  1.  0.  0.  1.  0.]] 	[ 210.]


In [8]:
# Run TF
np.random.seed(0)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for step in range(training_steps):
        _, loss_out = sess.run([training, loss], feed_dict={X: X_train, y:y_train})
        if step % display_steps == 0:
            print "Loss {} at timestep {} \n" .format(loss_out, step)
    # Interpretting the results on test set
    out = sess.run(logits, feed_dict={X: X_test})
    plot = True 
    if plot is True:
        print "Ground Truth \t Predicted"
        for i in range(testing_samples):
            print y_test[i], "->", out[i]

Loss 21405.0449219 at timestep 0 

Loss 0.652780413628 at timestep 5000 

Loss 0.425802528858 at timestep 10000 

Loss 0.271017998457 at timestep 15000 

Loss 0.221777930856 at timestep 20000 

Loss 0.113618701696 at timestep 25000 

Loss 0.0528116226196 at timestep 30000 

Loss 0.0494224913418 at timestep 35000 

Loss 0.0691263154149 at timestep 40000 

Loss 0.0905301049352 at timestep 45000 

Ground Truth 	 Predicted
[ 147.] -> [ 146.93043518]
[ 48.] -> [ 47.8298378]
[ 34.] -> [ 33.78536606]
[ 245.] -> [ 244.96083069]
[ 48.] -> [ 47.8298378]
[ 10.] -> [ 9.8763113]
[ 162.] -> [ 161.7947998]
[ 244.] -> [ 243.83790588]
[ 72.] -> [ 72.14963531]
[ 211.] -> [ 211.0690918]
[ 159.] -> [ 158.5680542]
[ 88.] -> [ 87.89588928]
[ 92.] -> [ 91.83252716]
[ 228.] -> [ 228.25613403]
[ 251.] -> [ 250.83132935]
[ 162.] -> [ 161.7947998]
[ 2.] -> [ 1.80910587]
[ 57.] -> [ 56.99879456]
[ 122.] -> [ 121.90625]
[ 135.] -> [ 134.87109375]


In [9]:
# Evaluation Metric
# 1 if Prediction is close to ground truth (np.round)
out_r = np.round_(out, decimals=0)
acc = out_r == y_test
acc = acc.sum()/float(len(y_test))
print "Accuracy is {}" .format(acc)

Accuracy is 1.0
